# Precision and Recall (Sensitivity)

https://medium.com/@nghihuynh_37300/understanding-evaluation-metrics-in-medical-image-segmentation-d289a373a3f

Basieren (auch wie Accuracy, F1 score und IoU) auf der Berechnung einer Konfusionsmatrix. Diese Matrix enthält die Anzahl der True Positives (TP), False Positives (FP), True Negatives (TN) und False Negatives (FN). Die Wertebereiche aller vorgestellten Metriken liegen zwischen 0 (schlecht) und 1 (optimal).

<img src="https://glassboxmedicine.com/wp-content/uploads/2019/02/confusion-matrix.png?w=431&h=243" alt="drawing" width="400"/>

In unserem Fall soll der überschwemmte Bereich segmentiert werden, wobei 1 für überschemmt/Wasser und 0 für Hintergrund steht:

TP: Wasser-Pixel korrekt als Wasser erkannt
FP: Hintergrund fälschlich als Wasser erkannt
FN: Wasser fälschlich als Hintergrund erkannt
TN: Hintergrund korrekt als Hintergrund erkann

## Precision

Precision ist der Anteil der korrekt als positiv erkannten Ergebnisse an allen als positiv vorhergesagten: 
$$
\text{Precision} = \frac{TP}{TP + FP}
$$

## Recall

Recall (auch Sensitivität) ist der Anteil der korrekt erkannten positiven Fälle an allen tatsächlich positiven Fällen:
$$
\text{Recall} = \frac{TP}{TP + FN}
$$

In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K
import keras

@keras.saving.register_keras_serializable()
def precision_metric(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)

    tp = K.sum(y_true * y_pred)
    fp = K.sum(y_pred) - tp

    precision = tp / (tp + fp + K.epsilon())
    return precision

@keras.saving.register_keras_serializable()
def recall_metric(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)

    tp = K.sum(y_true * y_pred)
    fn = K.sum(y_true) - tp

    recall = tp / (tp + fn + K.epsilon())
    return recall


2025-06-09 19:50:30.978121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749498630.997131 1743300 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749498631.002974 1743300 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749498631.017410 1743300 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749498631.017430 1743300 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749498631.017431 1743300 computation_placer.cc:177] computation placer alr

`flatten`: sklearn.metrics akzeptiert nur 1D-Vektoren und keine 2D-Matrizen wie Segmentierungsmasken. `.flatten()` wandelt das Bild in eine Liste von Pixelwerten um.

# Dice Coefficient (F1-Score)

https://medium.com/@nghihuynh_37300/understanding-evaluation-metrics-in-medical-image-segmentation-d289a373a3f

Der F1-Score (auch Dice-Koeffizient) eignet sich für die Bewertung von Computer Vision und somit auch für die Segmentierung. 

Er misst die Überlappung zwischen Vorhersage und Ground Truth, basiert auf Precision und Recall und bestraft falsche Positive, was bei unausgeglichenen Datensätzen wichtig ist.

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*tSqwQ9tvLmeO9raDqg3i-w.png" alt="drawing" width="400"/>


Im Vergleich zur IoU ist der Dice-Koeffizient weniger streng bei Unter- oder Übersegmentierung.

Formel des F1-Score:

$$
\text{F1} = \frac{2TP}{2TP + FP + FN} = 2 * \frac{Precision * Recall }{Precision + Recall}
$$

In [2]:
import tensorflow as tf
from tensorflow.keras import backend as K
import keras

@keras.saving.register_keras_serializable()
def f1_metric(y_true, y_pred):
    precision = precision_metric(y_true, y_pred)
    recall = recall_metric(y_true, y_pred)
    return 2 * (precision * recall) / (precision + recall + K.epsilon())
